api call

In [29]:
import random
from openai import OpenAI
from tqdm import tqdm
import httpx
import pandas as pd
import openai
import backoff

api_key = "sk-mE91TMZY8yikxpif8fBa64F0BaBa4d76BcCdD0Cb13F437D2"
client = OpenAI(
    base_url="https://oneapi.xty.app/v1",  # 中转url
    api_key=api_key,                      # api_key
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)

@backoff.on_exception(backoff.expo, (openai.APIStatusError, openai.InternalServerError), max_tries=5)
def get_responce(messages):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        temperature=0.0,
    )
    return response.choices[0].message.content.strip('\n')
    # return response


logs = '''10.11.10.1 "GET /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/detail HTTP/1.1" status: 200 len: 1893 time: 0.2477829'''


messages = [
    {'role': 'system',
        'content': '''You will be provided with a log message. You should Replace variables with `<*>` to extract the corresponding template. The variables might be numbers, strings(name, url, path, data and time), or other types of data.
        Print the input log's template delimited by backticks.'''},
    {'role': 'user', 'content': 'log message:\n' + logs},
]
# 
print(get_responce(messages))

`10.11.10.1 "GET <*> HTTP/1.1" status: <*> len: <*> time: <*>`


In [6]:
from parsing_copy import get_candidate_template
logs = ['connection from 210.245.165.136 () at Wed Jun 22 13:16:30 2005']
templates = ['connection from <*> (<*>) at <*>', 'Kerberos authentication failed',
             'input_userauth_request: invalid user  [preauth]']
print(get_candidate_template(templates, logs))

('connection from <*> (<*>) at <*>', 0.7213114754098361)


In [7]:
import re

log = "connection from 203.101.45.59 () at Sun Jul  3 10:05:25 2005"
template = "connection from <*> () at <*>"

# 将模板中的"<*>"替换为正则表达式的通配符".*?"
template = template.replace("<*>", "(.*?)")

# 使用正则表达式匹配日志字符串
match = re.search(template, log)

# 提取匹配的字符串
if match:
    print(match.groups())

Evaluate_gpt

In [1]:
from evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
m,n,p,q = [],[],[],[]
for dataset in datasets:
    file = f'outputs/parser/Test/{dataset}.csv'  # Fifth_=_0.1
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a,b,c,d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print('     average: group Accuracy: %.4f, Message-Level Accuracy: %.4f, Edit Distance: %.4f' % (sum(m)/len(m), sum(n)/len(n), sum(p)/len(p)))

# 81.0 71.2

        HDFS: group Accuracy: 1.0000, Message-Level Accuracy: 0.9425, Edit Distance: 0.0575
      Hadoop: group Accuracy: 0.9850, Message-Level Accuracy: 0.8705, Edit Distance: 7.0345
       Spark: group Accuracy: 0.9225, Message-Level Accuracy: 0.8805, Edit Distance: 2.6105
   Zookeeper: group Accuracy: 0.9685, Message-Level Accuracy: 0.6510, Edit Distance: 2.3625
         BGL: group Accuracy: 0.9830, Message-Level Accuracy: 0.9615, Edit Distance: 0.5625
         HPC: group Accuracy: 0.9345, Message-Level Accuracy: 0.9360, Edit Distance: 0.3760
 Thunderbird: group Accuracy: 0.9750, Message-Level Accuracy: 0.9245, Edit Distance: 0.9690
     Windows: group Accuracy: 0.9950, Message-Level Accuracy: 0.7465, Edit Distance: 2.7875
       Linux: group Accuracy: 0.5645, Message-Level Accuracy: 0.6030, Edit Distance: 3.0035
     Android: group Accuracy: 0.8125, Message-Level Accuracy: 0.6690, Edit Distance: 13.3960
   HealthApp: group Accuracy: 0.6610, Message-Level Accuracy: 0.5275, Edit Dist

In [ ]:
# discard the target dataset
dataset = 'Apache'
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac',
            'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
datasets.remove(dataset)
global demonstration_templates
global demonstration_logs
demonstration_templates = []
demonstration_logs = []
for d in datasets:
    df = pd.read_csv(f'dataset\{d}\{d}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template  in zip(list1, list2):
        if template not in demonstration_templates:
            demonstration_templates.append(template)
            demonstration_logs.append(log)


In [ ]:
import difflib


def find_closest_matches(target, list1, list2, n=5):
    matches = difflib.get_close_matches(target, list1, n=n,cutoff=0.1)
    indices = [list1.index(match) for match in matches]
    return [list1[index] for index in indices],[list2[index] for index in indices]

list1, list2 = find_closest_matches('PacketResponder 1 for block blk_38865049064139660 terminating', demonstration_templates, demonstration_logs)

print(list1)
print(list2)

['PacketResponder <*> for block <*> terminating', 'IPC Server Responder: starting', 'Starting Socket Reader #<*> for port <*>', 'getAllCellInfo is blocked by permission manager . callingPackage = <*>', 'getCellLocation is blocked by permission manager uid:<*> pid:<*>']
['PacketResponder 1 for block blk_38865049064139660 terminating', 'IPC Server Responder: starting', 'Starting Socket Reader #1 for port 62260', 'getAllCellInfo is blocked by permission manager . callingPackage = com.tencent.mobileqq', 'getCellLocation is blocked by permission manager uid:10111 pid:28601']


In [ ]:
import re


def replace_numbers_with_zero(text):
    return re.sub(r'\d+(\.\d+)?', '0', text)


input = 'Failed password for root from 5.36.59.76 port 42393 ssh2'
print(replace_numbers_with_zero(input))
print(replace_numbers_with_zero(replace_numbers_with_zero(input)))

Failed password for root from 0.0 port 0 ssh0
Failed password for root from 0 port 0 ssh0


In [ ]:
from evaluator import evaluate
import pandas as pd
datasets = ['BGL', 'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC',
            'Zookeeper', 'Mac', 'Hadoop', 'Android', 'Windows', 'Apache', 'Thunderbird', 'Spark']
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# datasets = ['OpenSSH']
m, n, p, q = [], [], [], []
for dataset in datasets:
    file = f'outputs/parser/Fifth_=_0.1/{dataset}.csv'  # Fourth_guding
    # df = pd.read_csv(f'outputs/k_means/initial/{dataset}.csv')
    # df2 =
    a, b, c, d = evaluate(file, dataset)
    m.append(a)
    n.append(b)
    p.append(c)
    q.append(d)

print(sum(m)/len(m))
print(sum(n)/len(n))
print(sum(p)/len(p))

# 81.0 71.2

HDFS: group Accuracy: 1.0000, Message-Level Accuracy: 0.9600, Edit Distance: 0.0400
Hadoop: group Accuracy: 0.9850, Message-Level Accuracy: 0.6280, Edit Distance: 8.0760
Spark: group Accuracy: 0.9225, Message-Level Accuracy: 0.8970, Edit Distance: 1.9730
Zookeeper: group Accuracy: 0.9925, Message-Level Accuracy: 0.6425, Edit Distance: 1.6640
BGL: group Accuracy: 0.9595, Message-Level Accuracy: 0.9285, Edit Distance: 1.9975
HPC: group Accuracy: 0.8720, Message-Level Accuracy: 0.7115, Edit Distance: 2.0475
Thunderbird: group Accuracy: 0.9705, Message-Level Accuracy: 0.9075, Edit Distance: 0.9310
Windows: group Accuracy: 1.0000, Message-Level Accuracy: 0.8530, Edit Distance: 1.7600
Linux: group Accuracy: 0.6255, Message-Level Accuracy: 0.6650, Edit Distance: 3.1285
Android: group Accuracy: 0.9460, Message-Level Accuracy: 0.6550, Edit Distance: 10.1905
HealthApp: group Accuracy: 0.7805, Message-Level Accuracy: 0.6495, Edit Distance: 9.7320
Apache: group Accuracy: 1.0000, Message-Level Accu

In [ ]:
dataset = 'Linux'
df = pd.read_csv(
    f"dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv")
list = df['EventTemplate'].tolist()
templates = []
for item in list:
    if item not in templates:
        templates.append(item)
print(len(templates))
for template in templates:
    print(template)


115
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>
check pass; user unknown
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=<*>  user=<*>
session opened for user <*> by (uid=<*>)
session closed for user <*>
ALERT exited abnormally with [<*>]
connection from <*> (<*>) at <*>
User unknown timed out after <*> seconds at <*>
cupsd shutdown succeeded
cupsd startup succeeded
restart.
connection from <*> () at <*>
Received SNMP packet(s) from <*>
Authentication failed from <*> (<*>): Permission denied in replay cache code
Kerberos authentication failed
Authentication failed from <*> (<*>): Software caused connection abort
*** info [mice.c(<*>)]:
<*>: Auto-detected intellimouse <*>
session opened for user <*> by LOGIN(uid=<*>)
ROOT LOGIN ON <*>
removing device node '<*>'
creating device node '<*>'
authentication failure; logname= uid=<*> euid=<*> tty=<*> ruser= rhost=
Couldn't authenticate user
ANONYMOUS FTP LOGIN FROM <*>,  (anonymous)
notify

验证按' '分词有多少词，以及含数字的词占多少

In [ ]:
import re
import pandas as pd
from post_process import correct_single_template
datasets = ['BGL' ,'HDFS', 'Linux', 'HealthApp', 'OpenStack', 'OpenSSH', 'Proxifier', 'HPC', 'Zookeeper', 'Mac','Android','Hadoop', 'Windows', 'Apache', 'Thunderbird', 'Spark']
# datasets = ['Linux']


def tokenize(log_content, tokenize_pattern=r'[ ,]'):
    words = re.split(tokenize_pattern, log_content)
    for index, word in enumerate(words):
        if word.startswith('/') and len(word) > 1:
            words[index] = ''
        if '=' in word:
            words[index] = word.split('=')[0]
        if re.search(r'\d', word):
            words[index] = ''
    words = [word for word in words if word]   # remove null
    return words


a = 0
b = 0
list = []
list2 = []
k = 0
temp = ['structured', 'templates', 'Content', 'EventTemplate']
tokens = []
for dataset in datasets:
    df = pd.read_csv(f"dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv")
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()

    print('-' * 20)
    print(dataset)

    # 验证长度为1-3的tempalte是否含有<*>
    # NOTE: 长度为1时，不用解析，2-3仍然需要解析
    # DS
    # NOTE: 没有template中含有'  '
    # BL+US
    # NOTE: boolen变量主要出现在Android中 null:Mac, Android root:many datasets admin:OpenSSH, Thunderbird
    # BL = ['true', 'false']
    # US = ['null', 'root', 'admin']
    # DG
    # NOTE: 没有template含有纯数字
    #
    tem = []
    for log, template in zip(logs, templates):
        if '/' in log and template not in tem:
            words = tokenize(log)
            tem.append(template)
            print(words)
            print(template)
            print('-' * 20)
            
            # print('-' * 20)
    # 验证按' '分词有多少词，以及含数字的词占多少
    # list2.append(len(list))    
    # total_words = 0
    # words_with_numbers = 0

    # for s in list:
    #     words = s.split()
    #     total_words += len(words)
    #     words_with_numbers += len([word for word in words if re.search(r'\d', word)])
    # a+=total_words
    # b+=words_with_numbers


# print(f"{b} / {a}")

--------------------
BGL
['force', 'load/store']
force load/store alignment...............<*>
--------------------
['ciod:', 'LOGIN', 'failed:', 'No', 'such', 'file', 'or', 'directory']
ciod: LOGIN chdir(<*>) failed: No such file or directory
--------------------
['ciod:', 'Error', 'loading', 'invalid', 'or', 'missing', 'program', 'image', 'No', 'such', 'file', 'or', 'directory']
ciod: Error loading <*>: invalid or missing program image, No such file or directory
--------------------
['ciod:', 'Error', 'creating', 'node', 'map', 'from', 'file', 'No', 'child', 'processes']
ciod: Error creating node map from file <*>: No child processes
--------------------
['ciod:', 'Error', 'creating', 'node', 'map', 'from', 'file', 'Bad', 'file', 'descriptor']
ciod: Error creating node map from file <*>: Bad file descriptor
--------------------
['ciod:', 'Missing', 'or', 'invalid', 'fields', 'on', 'line', 'of', 'node', 'map', 'file']
ciod: Missing or invalid fields on line <*> of node map file <*>
---

GPT-4 call

In [ ]:
from openai import OpenAI
import httpx
client = OpenAI(
    base_url="https://4.0.996444.icu/v1", 
    api_key="sk-1XhqpmIwxu9nCuCD51FcA00948F54eDf9cA4765998Ce8f0d",
    http_client=httpx.Client(
        proxies="http://127.0.0.1:7890"  # 代理地址
    ),
)
message_list = [
     {"role": "user", "content": "Hello"}, 
    ]

response = client.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=message_list,
    temperature=0,
    max_tokens=1024
)

print(response)
response = response.choices[0].message.content

ChatCompletion(id='chatcmpl-8iJQI0nqY3S77kl5nwcKQ8rfZHNc4', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1705571786, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=8, total_tokens=17))


In [ ]:

import re
tokenize_pattern = r'[#= ,]'
log_content = 'xy=123,z=789,7#8#9 sex you '
words = re.split(tokenize_pattern, log_content)
print(words)

['xy', '123', 'z', '789', '7', '8', '9', 'sex', 'you', '']


In [ ]:
import re


def tokenize(log_content, tokenize_pattern=r'[ ,]'):
    words = re.split(tokenize_pattern, log_content)
    for index, word in enumerate(words):
        if word.startswith('/') and len(word) > 1:
            words[index] = ''
        if '=' in word:
            words[index] = word.split('=')[0]
        if re.search(r'\d', word):
            words[index] = ''
    words = [word for word in words if word]   # remove null
    return words





# root, admin, 









logs = []


logs.append(
    'proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01')
logs.append(
    'proxy.cse.cuhk.edu.hk:5070 close, 451 bytes sent, 18846 bytes (18.4 KB) received, lifetime <1 sec')
logs.append('proxy.cse.cuhk.edu.hk:5070 close, 1165 bytes (1.13 KB) sent, 3098 bytes (3.02 KB) received, lifetime 00:01')

for log in logs:







    print(tokenize(log))

['close', 'bytes', 'sent', 'bytes', 'received', 'lifetime']
['close', 'bytes', 'sent', 'bytes', 'KB)', 'received', 'lifetime', 'sec']
['close', 'bytes', 'KB)', 'sent', 'bytes', 'KB)', 'received', 'lifetime']


Drain

In [ ]:
from drain3 import TemplateMiner

# 创建一个模板挖掘器实例
miner = TemplateMiner()


# 添加一些日志行
logs = list(set(prompt.split('\n')))
for log in logs:
    result = miner.add_log_message(log)
    if result is not None:
        print(f"New template: {result}")

# 打印所有的日志模板
print("Log templates:")
for template in miner.drain.clusters:
    print(template.get_template())

config file not found: drain3.ini


New template: {'change_type': 'cluster_created', 'cluster_id': 1, 'cluster_size': 1, 'template_mined': 'authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= rhost=209.152.168.249 user=guest', 'cluster_count': 1}
New template: {'change_type': 'cluster_template_changed', 'cluster_id': 1, 'cluster_size': 2, 'template_mined': 'authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= <*> user=guest', 'cluster_count': 1}
Log templates:
authentication failure; logname= uid=0 euid=0 tty=NODEVssh ruser= <*> user=guest


In [ ]:
%pip install Levenshtein

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/101.2 kB ? eta -:--:--
     -------------------------------------- 101.2/101.2 kB 5.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     ---------------- ----------------------- 0.7/1.6 MB 14.4 MB/s eta 0:00:01
     ------------------------------------- -- 1.5/1.6 MB 14.1 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 13.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from Levenshtein import distance


def closest_template(target, templates):
    min_distance = float('inf')
    closest_template = None
    for template in templates:
        d = distance(target, template)
        if d < min_distance:
            min_distance = d
            closest_template = template
    return closest_template

templates = [
    '<*> close, <*> bytes sent, <*> bytes received, lifetime <1',
    'proxy.cse.cuhk.edu.hk:<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1',
    '<*> close, <*> bytes sent, <*> bytes (<*> KB) received, lifetime <*>',
    '<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> MB) received, lifetime <*>',
]

print(closest_template('proxy.cse.cuhk.edu.hk:5070 close, 0 bytes sent, 0 bytes received, lifetime 00:01', templates))

proxy.cse.cuhk.edu.hk:<*> close, <*> bytes (<*> KB) sent, <*> bytes (<*> KB) received, lifetime <1


生成候选的labelled logs

In [ ]:
import json
import pandas as pd
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')

data = {}
templates = []
logs = []
for dataset in datasets:
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    list1 = df['Content'].tolist()
    list2 = df['EventTemplate'].tolist()
    for log, template  in zip(list1, list2):
        if template not in templates:
            templates.append(template)
            logs.append(log)
    # 'logs': logs_c, 'templates': templates_c,
    print(dataset, len(logs))


HDFS 14
Hadoop 128
Spark 164
Zookeeper 214
BGL 334
HPC 380
Thunderbird 529
Windows 579
Linux 687
Android 845
HealthApp 920
Apache 926
Proxifier 934
OpenSSH 960
OpenStack 1003
Mac 1344


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(target, lst, n=3):
    vectors = [np.array([ord(c) for c in s]).reshape(1, -1) for s in lst]
    target_vector = np.array([ord(c) for c in target]).reshape(1, -1)
    similarities = cosine_similarity(vectors, target_vector)
    most_similar_indices = np.argsort(similarities, axis=0)[-n:].flatten()[::-1]
    return [lst[i] for i in most_similar_indices]

In [4]:
import pandas as pd
import re
# read log
table_order = 'HDFS Hadoop Spark Zookeeper BGL HPC Thunderbird Windows Linux Android HealthApp Apache Proxifier OpenSSH OpenStack Mac'
datasets = table_order.split(' ')
# define a set
list = []
for dataset in datasets:
    print(dataset)
    print('-' * 20)
    df = pd.read_csv(f'dataset\{dataset}\{dataset}_2k.log_structured_corrected.csv')
    logs = df['Content'].tolist()
    templates = df['EventTemplate'].tolist()
    for log,template in zip(logs, templates):    
        if ':' in log and template not in list:
            print(log)
            list.append(template)


HDFS
--------------------
BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
Receiving block blk_5792489080791696128 src: /10.251.30.6:33145 dest: /10.251.30.6:50010
BLOCK* NameSystem.allocateBlock: /user/root/rand/_temporary/_task_200811092030_0001_m_000590_0/part-00590. blk_-1727475099218615100
10.251.194.213:50010 Served block blk_-7724713468912166542 to /10.251.203.80
10.251.30.85:50010:Got exception while serving blk_-2918118818249673980 to /10.251.90.64:
BLOCK* NameSystem.delete: blk_2568309208894455676 is added to invalidSet of 10.251.31.160:50010
10.250.15.198:50010 Starting thread to transfer block blk_4292382298896622412 to 10.250.15.240:50010
BLOCK* ask 10.250.18.114:50010 to delete  blk_-5140072410813878235
Received block blk_-4411589101766563890 src: /10.250.14.38:37362 dest: /10.250.14.38:50010 of size 67108864
BLOCK* ask 10.250.14.38:50010 to replicate blk_-7571492020523929240 to datanode(s) 10.251.12

In [4]:
list = [1,2,3,4]
for item in list:
    item = 1
print(list)

[1, 2, 3, 4]
